In [1]:
'''
module='gekko'
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
pipmain(['install',module])
'''


"\nmodule='gekko'\ntry:\n    from pip import main as pipmain\nexcept:\n    from pip._internal import main as pipmain\npipmain(['install',module])\n"

In [5]:
from gekko import GEKKO
import os

In [6]:
dir1 = os.path.abspath(os.curdir)

# Recherche d'une solution faisable 

In [7]:
# -*- coding: utf-8 -*-
"""
@author: Groupe formé de 
Akram IBN EL AHMER​
Hermann LEIBNITZ KLAUS AGOSSOU​
Reda ASSIOUI​
Olivier DANIEL STEPHANE KOMBA MBONGO​
Souad EZZOUINE​
"""
import os
import sys
import numpy as np
import json
from gekko import GEKKO






####################
### Utils ##########
####################

## Global variables
CUR_DIR = os.getcwd()
PAR_DIR = os.path.dirname(CUR_DIR)
RESOURCES_STR = 'Resources'
SEASONS_STR = 'Seasons'
INTERVENTIONS_STR = 'Interventions'
EXCLUSIONS_STR = 'Exclusions'
T_STR = 'T'
SCENARIO_NUMBER = 'Scenarios_number'
RESOURCE_CHARGE_STR = 'workload'
TMAX_STR = 'tmax'
DELTA_STR = 'Delta'
MAX_STR = 'max'
MIN_STR = 'min'
RISK_STR = 'risk'
START_STR = 'start'
QUANTILE_STR = "Quantile"
ALPHA_STR = "Alpha"

## Json reader
def read_json(filename):
    """Read a json file and return data as a dict object"""

    print('Reading json file ' + filename + '...')
    f = open(filename, 'r')
    Instance = json.load(f)
    f.close()
    print('Done')
    #print("Instance = ",Instance)
    return Instance


os.chdir(dir1)


def resolution(instance_file,directory,base):
    # Read Instance
    Instance = read_json(instance_file)
    os.chdir(directory)
    m = GEKKO(remote=False)
    nj = Instance['T'] #nb de start_time 
    x = {}
    
    Interventions = Instance[INTERVENTIONS_STR]
    
    #############################################################
    ########################variables############################
    #############################################################
    
    
    #definie les variables 
    for intervention_name, intervention in Interventions.items():
        x[intervention_name] = [m.Var(lb=0,ub=1,integer=True) for j in range(nj)]
    #les stocker dans le dictionnaire 
    for intervention_name, intervention in Interventions.items():
        start_time = sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ])
        Interventions[intervention_name][START_STR] = start_time
        
    #############################################################
    ########################contraintes##########################
    #############################################################
    #contraintes 
        #for each intervention: 
            #- 1 <= start_time <= horizon_end=instance["T"]
            #- int(intervention[TMAX_STR]) >= start_time
    
    """ schedule constraints"""
    #contrainte 1 #qq i, 1 <= start_time <= horizon_end=instance["T"]
    Interventions = Instance[INTERVENTIONS_STR]
    for intervention_name, intervention in Interventions.items():
        #il y a doit y avoir un start time  
        horizon_end = Instance[T_STR] 
        
        # cdt #qq i, 1 <= start_time <= horizon_end=instance["T"]
        s = m.Intermediate(0)
        for st in range(1,horizon_end+1):
            s+= x[intervention_name][st-1]
        m.Equation(s==1)
        #j'ajoute apres cette contrainte à travers une contrainte de type inegalite
        
        #cdt #qq i, 1 <= start_time + delta_i_t<= horizon_end=instance["T"] +1
        # ... retrieve their respective starting times...
        start_time = sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ]) 
        # ... and their respective deltas (duration)
        intervention_delta = sum( [x[intervention_name][st-1]*int(intervention[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention[TMAX_STR])+1) ]) # get index in list
        m.Equation( start_time + intervention_delta<= Instance[T_STR]+1)
        m.Equation (  start_time <= intervention[TMAX_STR] ) #contrainte2 #qq i, int(intervention[TMAX_STR]) >= start_time
        
    #contrainte2 #qq i, int(intervention[TMAX_STR]) >= start_time
    for intervention_name, intervention in Interventions.items():
        start_time = sum([x[intervention_name][st-1] * st for st in range(1,int(intervention[TMAX_STR])+1) ])
        m.Equation (  start_time <= intervention[TMAX_STR] )
    
    
    """resources constraints"""
    
    print('\tresources constraints...')
    #contrainte3 
    horizon_end = Instance[T_STR]
    """Compute effective workload (i.e. resources consumption values) for every resource and every time step"""
            
    tolerance=1e-5
    #print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    for c, constraint in Instance[RESOURCES_STR].items():
        print("ressource = ",c)
        for t in range(1,horizon_end+1):
            ss=0
            print("c = {} et t = {}".format(c,t))
            for intervention_name, intervention in Interventions.items():
                for st in range(1,int(intervention[TMAX_STR])+1):
                    if c in intervention["workload"] and str(t) in intervention["workload"][c] and str(st) in intervention["workload"][c][str(t)]:
                        a = x[intervention_name][st-1]
                        #print("t = {} et st = {} et interv = {} et ressource = {}".format(t,st,intervention_name,c))
                        b = intervention["workload"][c][str(t)][str(st)]
                        ss+=a*b
                        #ss+= x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    
            m.Equation(ss>= constraint[MIN_STR][t-1] - tolerance )
            m.Equation(constraint[MAX_STR][t-1] + tolerance >= ss)
            

    """exclusions constraints"""
    
    print('exclusions constraints...')
    # Retrieve Interventions and Exclusions
    Interventions = Instance[INTERVENTIONS_STR]
    Exclusions = Instance[EXCLUSIONS_STR]
    # Assert every exclusion holds
    for exclusion in Exclusions.values():
        # Retrieve exclusion infos
        [intervention_1_name, intervention_2_name, season] = exclusion
        # Retrieve concerned interventions...
        intervention_1 = Interventions[intervention_1_name]
        intervention_2 = Interventions[intervention_2_name]
        # ... their respective starting times...
        intervention_1_start_time = sum ([ (st+1)* x[intervention_1_name][st] for st in range( len(x[intervention_1_name])) ])
        intervention_2_start_time = sum ([ (st+1)* x[intervention_2_name][st] for st in range( len(x[intervention_2_name])) ])
        # ... and their respective deltas (duration)
        intervention_1_delta = sum( [x[intervention_1_name][st-1]*int(intervention_1[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_1[TMAX_STR])+1) ]) # get index in list
        #intervention_1_delta = int(intervention_1[DELTA_STR][intervention_1_start_time - 1]) # get index in list
        intervention_2_delta = sum( [x[intervention_2_name][st-1]*int(intervention_2[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_2[TMAX_STR])+1) ]) # get index in list
        # Check overlaps for each time step of the season 
        for time_str in Instance[SEASONS_STR][season]:
            time = int(time_str)
            #equation
            #if (intervention_1_start_time <= time < intervention_1_start_time + intervention_1_delta) and (intervention_2_start_time <= time < intervention_2_start_time + intervention_2_delta): pbb"""
            
            #si (intervention_1_start_time <= time, cdt1_1 = 1
            cdt1_1 = m.if2(-intervention_1_start_time + time,0,1)
            #si time < intervention_1_start_time + intervention_1_delta, cdt1_2=1
            cdt1_2 = m.if2(time - intervention_1_start_time + intervention_1_delta,1,0)
            #si (intervention_2_start_time <= time, cdt2_1 = 1
            cdt2_1 = m.if2(-intervention_2_start_time + time,0,1)
            #si time < intervention_2_start_time + intervention_1_delta, cdt2_2=1
            cdt2_2 = m.if2(time - intervention_2_start_time + intervention_2_delta,1,0)

            cdt1 = cdt1_1 + cdt1_2 #est ==2 si t est dans l'intervalle
            cdt2 = cdt2_1 + cdt2_2 #est == 2 si t est dans l'intervalle
            m.Equation (cdt1 + cdt2 <4)
                

    print('\tok')
    
    
  
    #solution 
    m.options.SOLVER=1#pour des solututions entieres
    print("le max = ",m.options.MAX_ITER)
    #m.options.MAX_ITER = 100000
    m.options.MAX_MEMORY = 50000
    m.solve(disp=True,debug=True)
    #apm_get(server,app,'infeasibilities.txt')
    #m.open_folder()
    print(x)
    out = open( os.path.join(directory,'output_'+base+'.txt'),"w")
    for intervention, liste in x.items():
        for i in range(len(liste)):
            elt = liste[i]
            if int(elt[0])==1:
                print(intervention," : t",i+1 )
                out.writelines( [intervention +  ' ' + str(i+1)+'\n'])

    print("le max = ",m.options.MAX_ITER)
    
    out.close()
    
    
    

#prend le chemin vers le ichier json
def solus(path):
    os.chdir(dir1)
    instance_file = os.path.abspath(path)
    #print(1)
    #print("instance_file = ", instance_file[2:4], "type = ",type(instance_file))


    #creer un dossier dont le nom est celui diu fichier json 
    base=os.path.basename(instance_file)
    base = os.path.splitext(base)
    print("base1 = ", base[1])
    if base[1]!='json': print('ERROR: expecting a json file ')
    base  = base[0]
    directory = os.path.abspath(os.path.join('_SansObj_solver=1',base))
    if not os.path.exists(directory):
        os.makedirs(directory)


    #out = open( os.path.join(directory,'output_'+base+'.txt'),"w")
    
    resolution(instance_file,directory,base)
    
    
    os.chdir(dir1)
## sans objectif; no shutdown; solus, "AvecObj1"

In [8]:
#donner le chemin relatif ou absolu vers le fichier
solus("files\\example3.json")

('base1 = ', '.json')
ERROR: expecting a json file 
Reading json file C:\Users\hermann.agossou\Documents\docs\A_set\A_set\files\example3.json...
Done
	resources constraints...
('ressource = ', u'Ressources_1')
c = Ressources_1 et t = 1
c = Ressources_1 et t = 2
c = Ressources_1 et t = 3
c = Ressources_1 et t = 4
c = Ressources_1 et t = 5
c = Ressources_1 et t = 6
c = Ressources_1 et t = 7
c = Ressources_1 et t = 8
c = Ressources_1 et t = 9
c = Ressources_1 et t = 10
c = Ressources_1 et t = 11
c = Ressources_1 et t = 12
c = Ressources_1 et t = 13
c = Ressources_1 et t = 14
c = Ressources_1 et t = 15
c = Ressources_1 et t = 16
c = Ressources_1 et t = 17
('ressource = ', u'Resources_control')
c = Resources_control et t = 1
c = Resources_control et t = 2
c = Resources_control et t = 3
c = Resources_control et t = 4
c = Resources_control et t = 5
c = Resources_control et t = 6
c = Resources_control et t = 7
c = Resources_control et t = 8
c = Resources_control et t = 9
c = Resources_control 

In [9]:
'''
for i in range(1,10):
    try: solus("files\\A_0{}.json".format(i))
    except: pass
for i in range(10,16):
    try: solus("files\\A_{}.json".format(i))
    except: pass
'''

'\nfor i in range(1,10):\n    try: solus("files\\A_0{}.json".format(i))\n    except: pass\nfor i in range(10,16):\n    try: solus("files\\A_{}.json".format(i))\n    except: pass\n'

In [10]:
'''
for i in range(1,4):
    solus("files\\example{}.json".format(i))
'''

'\nfor i in range(1,4):\n    solus("files\\example{}.json".format(i))\n'

In [11]:
solus("files\\example3.json")

('base1 = ', '.json')
ERROR: expecting a json file 
Reading json file C:\Users\hermann.agossou\Documents\docs\A_set\A_set\files\example3.json...
Done
	resources constraints...
('ressource = ', u'Ressources_1')
c = Ressources_1 et t = 1
c = Ressources_1 et t = 2
c = Ressources_1 et t = 3
c = Ressources_1 et t = 4
c = Ressources_1 et t = 5
c = Ressources_1 et t = 6
c = Ressources_1 et t = 7
c = Ressources_1 et t = 8
c = Ressources_1 et t = 9
c = Ressources_1 et t = 10
c = Ressources_1 et t = 11
c = Ressources_1 et t = 12
c = Ressources_1 et t = 13
c = Ressources_1 et t = 14
c = Ressources_1 et t = 15
c = Ressources_1 et t = 16
c = Ressources_1 et t = 17
('ressource = ', u'Resources_control')
c = Resources_control et t = 1
c = Resources_control et t = 2
c = Resources_control et t = 3
c = Resources_control et t = 4
c = Resources_control et t = 5
c = Resources_control et t = 6
c = Resources_control et t = 7
c = Resources_control et t = 8
c = Resources_control et t = 9
c = Resources_control 

# Minimisation de l'objectif 

In [15]:
# -*- coding: utf-8 -*-
"""
@author: Groupe formé de 
Akram IBN EL AHMER​
Hermann LEIBNITZ KLAUS AGOSSOU​
Reda ASSIOUI​
Olivier DANIEL STEPHANE KOMBA MBONGO​
Souad EZZOUINE​
"""
import os
import sys
import numpy as np
import json
from gekko import GEKKO


os.chdir(dir1)



####################
### Utils ##########
####################

## Global variables
CUR_DIR = os.getcwd()
PAR_DIR = os.path.dirname(CUR_DIR)
RESOURCES_STR = 'Resources'
SEASONS_STR = 'Seasons'
INTERVENTIONS_STR = 'Interventions'
EXCLUSIONS_STR = 'Exclusions'
T_STR = 'T'
SCENARIO_NUMBER = 'Scenarios_number'
RESOURCE_CHARGE_STR = 'workload'
TMAX_STR = 'tmax'
DELTA_STR = 'Delta'
MAX_STR = 'max'
MIN_STR = 'min'
RISK_STR = 'risk'
START_STR = 'start'
QUANTILE_STR = "Quantile"
ALPHA_STR = "Alpha"

## Json reader
def read_json(filename):
    """Read a json file and return data as a dict object"""

    print('Reading json file ' + filename + '...')
    f = open(filename, 'r')
    Instance = json.load(f)
    f.close()
    print('Done')
    #print("Instance = ",Instance)
    return Instance





def resolution(instance_file,directory,base):
    # Read Instance
    Instance = read_json(instance_file)
    os.chdir(directory)
    m = GEKKO(remote=False)
    T=Instance[T_STR] #nb  de time steps 
    """
    # optional solver settings with APOPT
    m.solver_options = ['minlp_maximum_iterations 5000', \
                        # minlp iterations with integer solution
                        'minlp_max_iter_with_int_sol 100', \
                        # treat minlp as nlp
                        'minlp_as_nlp 1', \
                        # nlp sub-problem max iterations
                        'nlp_maximum_iterations 5000', \
                        # 1 = depth first, 2 = breadth first
                        'minlp_branch_method 1', \
                        # maximum deviation from whole number
                        'minlp_integer_tol 0.0005', \
                        # covergence tolerance
                        'minlp_gap_tol 0.0001']
    """
    #variables 
    #ni = 3 #nb d'interventions 
    nj = Instance['T'] #nb de start_time 
    x = {}
    
    Interventions = Instance[INTERVENTIONS_STR]
    
    #############################################################
    ########################variables############################
    #############################################################
    
    
    #definie les variables 
    for intervention_name, intervention in Interventions.items():
        x[intervention_name] = [m.Var(lb=0,ub=1,integer=True) for j in range(nj)]
    #les stocker dans le dictionnaire 
    for intervention_name, intervention in Interventions.items():
        start_time = sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ])
        Interventions[intervention_name][START_STR] = start_time
        
    #############################################################
    ########################contraintes##########################
    #############################################################
    #contraintes 
        #for each intervention: 
            #- 1 <= start_time <= horizon_end=instance["T"]
            #- int(intervention[TMAX_STR]) >= start_time
    
    """ schedule constraints"""
    #contrainte 1 #qq i, 1 <= start_time <= horizon_end=instance["T"]
    Interventions = Instance[INTERVENTIONS_STR]
    for intervention_name, intervention in Interventions.items():
        #il y a doit y avoir un start time  
        horizon_end = Instance[T_STR] 
        
        # cdt #qq i, 1 <= start_time <= horizon_end=instance["T"]
        s = m.Intermediate(0)
        for st in range(1,horizon_end+1):
            s+= x[intervention_name][st-1]
        m.Equation(s==1)
        #j'ajoute apres cette contrainte à travers une contrainte de type inegalite
        
        #cdt #qq i, 1 <= start_time + delta_i_t<= horizon_end=instance["T"] +1
        # ... retrieve their respective starting times...
        start_time = sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ]) 
        # ... and their respective deltas (duration)
        intervention_delta = sum( [x[intervention_name][st-1]*int(intervention[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention[TMAX_STR])+1) ]) # get index in list
        m.Equation( start_time + intervention_delta<= Instance[T_STR]+1)
        m.Equation ( 1<=start_time) #contrainte: il debute pas à st==0 #meilleure contraonte que l'egalite
        m.Equation (  start_time <= intervention[TMAX_STR] ) #contrainte2 #qq i, int(intervention[TMAX_STR]) >= start_time
        

    
    """resources constraints"""

    print('\tresources constraints...')
    #contrainte3 
    horizon_end = Instance[T_STR]
    """Compute effective workload (i.e. resources consumption values) for every resource and every time step"""
   
    # tolerance=1e-5
    # print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    # for c, constraint in Instance[RESOURCES_STR].items():
        # for t in range(1,horizon_end+1):
            # ss = sum ([ sum([ x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    # for st in range(1,intervention[TMAX_STR]+1) ])
                    # for intervention_name, intervention in Interventions.items() 
                    # ])
            # m.Equation(  constraint[MAX_STR][t-1] + tolerance  >= ss)
            # m.Equation( ss  >= constraint[MIN_STR][t-1] - tolerance )
            
    tolerance=1e-5
    #print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    for c, constraint in Instance[RESOURCES_STR].items():
        print("ressource = ",c)
        for t in range(1,horizon_end+1):
            ss=0
            print("c = {} et t = {}".format(c,t))
            for intervention_name, intervention in Interventions.items():
                for st in range(1,int(intervention[TMAX_STR])+1):
                    if c in intervention["workload"] and str(t) in intervention["workload"][c] and str(st) in intervention["workload"][c][str(t)]:
                        a = x[intervention_name][st-1]
                        
                        b = intervention["workload"][c][str(t)][str(st)]
                        ss+=a*b
                        #ss+= x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    
            m.Equation(ss>= constraint[MIN_STR][t-1] - tolerance )
            m.Equation(constraint[MAX_STR][t-1] + tolerance >= ss)
            
    #m.Equation(resource[MAX_STR][t-1] + tolerance  >= resources_usage[resource_name][t-1])


    """exclusions constraints"""
    print('exclusions constraints...')
    # Retrieve Interventions and Exclusions
    Interventions = Instance[INTERVENTIONS_STR]
    Exclusions = Instance[EXCLUSIONS_STR]
    # Assert every exclusion holds
    i = 0
    for exclusion in Exclusions.values():
        i+=1
        # Retrieve exclusion infos
        [intervention_1_name, intervention_2_name, season] = exclusion
        # Retrieve concerned interventions...
        intervention_1 = Interventions[intervention_1_name]
        intervention_2 = Interventions[intervention_2_name]
        # ... their respective starting times...
        intervention_1_start_time = sum ([ (st+1)* x[intervention_1_name][st] for st in range( len(x[intervention_1_name])) ])
        intervention_2_start_time = sum ([ (st+1)* x[intervention_2_name][st] for st in range( len(x[intervention_2_name])) ])
        # ... and their respective deltas (duration)
        intervention_1_delta = sum( [x[intervention_1_name][st-1]*int(intervention_1[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_1[TMAX_STR])+1) ]) # get index in list
        #intervention_1_delta = int(intervention_1[DELTA_STR][intervention_1_start_time - 1]) # get index in list
        intervention_2_delta = sum( [x[intervention_2_name][st-1]*int(intervention_2[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_2[TMAX_STR])+1) ]) # get index in list
        # Check overlaps for each time step of the season 
        for time_str in Instance[SEASONS_STR][season]:
            time = int(time_str)
            #equation
            #"""if (intervention_1_start_time <= time < intervention_1_start_time + intervention_1_delta) and (intervention_2_start_time <= time < intervention_2_start_time + intervention_2_delta): pbb"""
            
            #si (intervention_1_start_time <= time, cdt1_1 = 1
            cdt1_1 = m.if2(-intervention_1_start_time + time,0,1)
            #si time < intervention_1_start_time + intervention_1_delta, cdt1_2=1
            cdt1_2 = m.if2(time - intervention_1_start_time + intervention_1_delta,1,0)
            #si (intervention_2_start_time <= time, cdt2_1 = 1
            cdt2_1 = m.if2(-intervention_2_start_time + time,0,1)
            #si time < intervention_2_start_time + intervention_1_delta, cdt2_2=1
            cdt2_2 = m.if2(time - intervention_2_start_time + intervention_2_delta,1,0)

            cdt1 = m.Intermediate(cdt1_1 + cdt1_2,name="firstexclusion_{}_{}_{}".format(i,time_str,intervention_1_name)) #est ==2 si t est dans l'intervalle
            cdt2 = m.Intermediate(cdt2_1 + cdt2_2,name="secondexclesion_{}_{}_{}".format(i,time_str,intervention_2_name)) #est == 2 si t est dans l'intervalle
            m.Equation (cdt1 + cdt2 <4)
                

    print('\tok')


    #############################################################
    ########################objectif#############################
    #############################################################
    """Compute objectives (mean and expected excess)"""
    
    ############objectif1   
    """Compute objectives (mean)"""
     
    horizon_end = Instance[T_STR]
    S = Instance[SCENARIO_NUMBER]  #nb de scenarios par time step 
    print("S = ",S)
    print("horizon_end = ",horizon_end)
    alpha = 0.5
    liste_risk_s_t = [[m.Intermediate(m.sum([m.sum(
                        [x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
                        for st in range(1,int(intervention[TMAX_STR])+1) if str(t) in intervention[RISK_STR] and str(st) in intervention[RISK_STR][str(t)]])
                        for intervention_name, intervention in Interventions.items() ]))
                        for sc in range(1,int(S[t-1])+1) ] for t in range(1,int(horizon_end)+1) ]
    obj1 = 0
    for t in range(1,horizon_end+1):
        obj1 += (1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) 
    obj1 = (1/T)*obj1 
    
    
    #obj1 = (1/T)* m.sum([ (1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) )]) 
    # ##
    # obj1 = (1/T)* m.sum([() for t in range(1,T+1)])        
    # () = (1/S[t])* m.sum([() for sc in range(1,S[t]) ])
    # () = m.sum([() for intervention_name, intervention in Interventions.items() ])
    # () = m.sum([( ) for st in range(1,T+1) ])
    # () = x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
    
    ##
    #m.Obj( obj1)
    
    
    
    
    """Compute risk distributions for all time steps, given the interventions starting times"""
    # liste_risk_s_t = [[m.sum([m.sum(
                        # [x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
                        # for st in range(1,int(intervention[TMAX_STR])+1) ])
                        # for intervention_name, intervention in Interventions.items() ])
                        # for sc in range(1,S[t-1]+1) ] for t in range(1,horizon_end+1) ]

    ############objectif2
    
    """Compute objectives (mean and expected excess)"""
    
    obj2  = 0
    Q_t = 0
    """
    for t in range(1,horizon_end+1):
        # quantile #le piem element quand on range l'ensemble dans l'odre croissant
            #calcul de p
        p = Instance[QUANTILE_STR]
        p = p*S[t-1]
        if p-int(p)!=0: 
            p = int(p)+1
        
        print("p = ",p)
        p = int(p)
        #index = p-1
            # trouver le p_ieme min 
        print("len() = ",S[t-1]+1)
        E = [ liste_risk_s_t[t-1][sc-1] for sc in range(1,S[t-1]+1)  ]
        print("le tri")
        for i in range (p):
            l_new = [elt for elt in E] 
            min = E[i]
            pos_min = i
            for j in range(i, len(E)):
                temp = m.if3( E[j]-min,E[j],min)
                min = temp
                temp = m.if3( E[j]-min,j,pos_min )
                pos_min = temp 
            # le min est trouve 
            # on echange les positions de i et pos_min 
            temp = E[j]
            E[i]=E[j] 
            E[j] = temp 
            #E[i], E[j] = E[j],E[i] 
            
        Q_t = E[p-1]
    
            
            
        

        print("Q_t = ",Q_t)
        
        
        
        #Compute risk distributions for all time steps, given the interventions starting times
        risk_t= (1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) 
        
        #calul de l'objectif 2
        obj2 += m.max2(0, Q_t - risk_t)
    """
    obj2 = (1/T)*obj2
    
    print("obj ",obj2)
    
    m.Obj(alpha*obj1 + (1-alpha)*obj2)
    
    #solution 
    m.options.SOLVER=1#pour des solututions entieres
    print("le max = ",m.options.MAX_ITER)
    #m.options.MAX_ITER = 100000
    m.options.MAX_MEMORY = 50000
    m.solve(disp=True,debug=True)
    #apm_get(server,app,'infeasibilities.txt')
    #m.open_folder()
    print(x)
    out = open( os.path.join(directory,'output_'+base+'.txt'),"w")
    for intervention, liste in x.items():
        for i in range(len(liste)):
            elt = liste[i]
            if int(elt[0])==1:
                print(intervention,"debut:t= {}".format(i+1 ) )
                out.writelines( [intervention +  ' ' + str(i+1)+'\n'])
    
    print("le max = ",m.options.MAX_ITER)
    
    out.close()
    

#prend le chemin vers le ichier json
def solus2(path):
    os.chdir(dir1)
    instance_file = os.path.abspath(path)
    #print(1)
    #print("instance_file = ", instance_file[2:4], "type = ",type(instance_file))


    #creer un dossier dont le nom est celui diu fichier json 
    base=os.path.basename(instance_file)
    base = os.path.splitext(base)
    if base[1]!='json': print('ERROR: expecting a json file ')
    base  = base[0]
    directory = os.path.abspath(os.path.join('_AvecObj1_solver1',base))
    if not os.path.exists(directory):
        os.makedirs(directory)


    
    
    resolution(instance_file,directory,base)
    
    os.chdir(dir1)

## Avec objectif1; no shutdown; solus2, "AvecObj1"

In [17]:
solus2("files\\example1.json")
solus2("files\\example3.json")
#solus2("files\\A_01.json")

ERROR: expecting a json file 
Reading json file C:\Users\hermann.agossou\Documents\docs\A_set\A_set\files\example1.json...
Done
	resources constraints...
('ressource = ', u'c1')
c = c1 et t = 1
c = c1 et t = 2
c = c1 et t = 3
exclusions constraints...
	ok
('S = ', [3, 3, 3])
('horizon_end = ', 3)
('obj ', 0)
('le max = ', 250)
 ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  51
   Constants    :  0
   Variables    :  177
   Intermediates:  18
   Connections  :  153
   Equations    :  139
   Residuals    :  121
 
 Number of state variables:    201
 Number of total equations: -  171
 Number of slack variables: -  18
 ---------------------------------------
 Degrees of freedom       :    12
 
 ----------------------------------------------
 Steady State Optimization 

In [18]:
print('done')

done


# Pour des problèmes de plus grandes tailles, desactiver des contraintes et utiliser un solver NLP 

In [19]:
# -*- coding: utf-8 -*-
"""
@author: Groupe formé de 
Akram IBN EL AHMER​
Hermann LEIBNITZ KLAUS AGOSSOU​
Reda ASSIOUI​
Olivier DANIEL STEPHANE KOMBA MBONGO​
Souad EZZOUINE​
"""
import os
import sys
import numpy as np
import json
from gekko import GEKKO


os.chdir(dir1)



####################
### Utils ##########
####################

## Global variables
CUR_DIR = os.getcwd()
PAR_DIR = os.path.dirname(CUR_DIR)
RESOURCES_STR = 'Resources'
SEASONS_STR = 'Seasons'
INTERVENTIONS_STR = 'Interventions'
EXCLUSIONS_STR = 'Exclusions'
T_STR = 'T'
SCENARIO_NUMBER = 'Scenarios_number'
RESOURCE_CHARGE_STR = 'workload'
TMAX_STR = 'tmax'
DELTA_STR = 'Delta'
MAX_STR = 'max'
MIN_STR = 'min'
RISK_STR = 'risk'
START_STR = 'start'
QUANTILE_STR = "Quantile"
ALPHA_STR = "Alpha"

## Json reader
def read_json(filename):
    """Read a json file and return data as a dict object"""

    print('Reading json file ' + filename + '...')
    f = open(filename, 'r')
    Instance = json.load(f)
    f.close()
    print('Done')
    #print("Instance = ",Instance)
    return Instance





def resolution(instance_file,directory,base):
    # Read Instance
    Instance = read_json(instance_file)
    os.chdir(directory)
    m = GEKKO(remote=False)
    T=int(Instance[T_STR]) #nb  de time steps 
    m.options.REDUCE = 3


    # optional solver settings with APOPT
    m.solver_options = ['minlp_maximum_iterations 100000', \
                        # minlp iterations with integer solution
                        'minlp_max_iter_with_int_sol 10', \
                        # nlp sub-problem max iterations
                        'nlp_maximum_iterations 50', \
                        # find integer solution faster
                        'minlp_branch_method 1'
                        # ecart par rapport à l'entier 
                        'minlp_integer_tol 0.2']
    
    #variables 
    #ni = 3 #nb d'interventions 
    nj = int(Instance['T']) #nb de start_time 
    horizon_end = T
    x = {}
    
    Interventions = Instance[INTERVENTIONS_STR]
    
    #############################################################
    ########################variables############################
    #############################################################
    
    
    #definie les variables 
    for intervention_name, intervention in Interventions.items():
        x[intervention_name] = [m.Var(lb=0,ub=1,integer=True,name="{}_t_{}".format(intervention_name,j+1)) for j in range(nj)]
    #les stocker dans le dictionnaire 
    for intervention_name, intervention in Interventions.items():
        Interventions[intervention_name][START_STR] = m.Intermediate(m.sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ]))
                                                                     
        
        
    #############################################################
    ########################contraintes##########################
    #############################################################
    #contraintes 
        #for each intervention: 
            #- 1 <= start_time <= horizon_end=instance["T"]
            #- int(intervention[TMAX_STR]) >= start_time
                                                                     
                                                                     
    """ schedule constraints
    """
    #contrainte 1 #qq i, 1 <= start_time <= horizon_end=instance["T"]
    Interventions = Instance[INTERVENTIONS_STR]
    for intervention_name, intervention in Interventions.items():
        #il y a doit y avoir un start time  
         
        
        # cdt #qq i, 1 <= start_time <= horizon_end=instance["T"]
        s = m.Intermediate(0)
        for st in range(1,horizon_end+1):
            s+= x[intervention_name][st-1]
        m.Equation(0.8<=s)
        m.Equation(s<=1.2)
        #j'ajoute apres cette contrainte à travers une contrainte de type inegalite
        
        #cdt #qq i, 1 <= start_time + delta_i_t<= horizon_end=instance["T"] +1
        # ... retrieve their respective starting times...
        start_time = m.Intermediate(m.sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ]) )
        # ... and their respective deltas (duration)
        '''intervention_delta = m.Intermediate( m.sum( [x[intervention_name][st-1]*int(intervention[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention[TMAX_STR])+1) ])) # get index in list
        '''
        #m.Equation( start_time + intervention_delta<= Instance[T_STR]+1)
        #m.Equation ( 1<=start_time) #contrainte: il debute pas à st==0 #meilleure contraonte que l'egalite
        m.Equation (  start_time <= intervention[TMAX_STR] ) #contrainte2 #qq i, int(intervention[TMAX_STR]) >= start_time
        
    
    
    """resources constraints"""

    print('\tresources constraints...')
    #contrainte3 
    #on desactive la contrainte sur les ressources
    
    """Compute effective workload (i.e. resources consumption values) for every resource and every time step"""
    '''
    # tolerance=1e-5
    # print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    # for c, constraint in Instance[RESOURCES_STR].items():
        # for t in range(1,horizon_end+1):
            # ss = sum ([ sum([ x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    # for st in range(1,intervention[TMAX_STR]+1) ])
                    # for intervention_name, intervention in Interventions.items() 
                    # ])
            # m.Equation(  constraint[MAX_STR][t-1] + tolerance  >= ss)
            # m.Equation( ss  >= constraint[MIN_STR][t-1] - tolerance )
            
    tolerance=1e-5
    #print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    for c, constraint in Instance[RESOURCES_STR].items():
        print("ressource = ",c)
        for t in range(1,horizon_end+1):
            ss=m.Intermediate(0)
            print("c = {}, t={}".format(c,t))
            for intervention_name, intervention in Interventions.items():
                
                for st in range(1,int(intervention[TMAX_STR])+1):
                    if c in intervention["workload"] and str(t) in intervention["workload"][c] and str(st) in intervention["workload"][c][str(t)]:
                        a =  m.Intermediate(x[intervention_name][st-1])
                        #print("t = {} et st = {} et interv = {} et ressource = {}".format(t,st,intervention_name,c))
                        b =  m.Intermediate(intervention["workload"][c][str(t)][str(st)])
                        ss+= m.Intermediate(a*b)
                        #ss+= x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    
            m.Equation(ss>= constraint[MIN_STR][t-1] - tolerance )
            #.Equation(constraint[MAX_STR][t-1] + tolerance >= ss)
            
    #m.Equation(resource[MAX_STR][t-1] + tolerance  >= resources_usage[resource_name][t-1])
    '''

    """exclusions constraints"""
    
    print('exclusions constraints...')
    # Retrieve Interventions and Exclusions
    Interventions = Instance[INTERVENTIONS_STR]
    Exclusions = Instance[EXCLUSIONS_STR]
    # Assert every exclusion holds
    for exclusion in Exclusions.values():
        # Retrieve exclusion infos
        [intervention_1_name, intervention_2_name, season] = exclusion
        # Retrieve concerned interventions...
        intervention_1 = Interventions[intervention_1_name]
        intervention_2 = Interventions[intervention_2_name]
        # ... their respective starting times...
        intervention_1_start_time =  m.Intermediate(m.sum ([ (st+1)* x[intervention_1_name][st] for st in range( len(x[intervention_1_name])) ]))
        intervention_2_start_time =  m.Intermediate( m.sum ([ (st+1)* x[intervention_2_name][st] for st in range( len(x[intervention_2_name])) ]))
        # ... and their respective deltas (duration)
        intervention_1_delta =  m.Intermediate(m.sum( [x[intervention_1_name][st-1]*int(intervention_1[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_1[TMAX_STR])+1) ])) # get index in list
        #intervention_1_delta = int(intervention_1[DELTA_STR][intervention_1_start_time - 1]) # get index in list
        intervention_2_delta =  m.Intermediate(m.sum( [x[intervention_2_name][st-1]*int(intervention_2[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_2[TMAX_STR])+1) ])) # get index in list
        # Check overlaps for each time step of the season 
        for time_str in Instance[SEASONS_STR][season]:
            time = int(time_str)
            #equation
            """if (intervention_1_start_time <= time < intervention_1_start_time + intervention_1_delta) and (intervention_2_start_time <= time < intervention_2_start_time + intervention_2_delta): pbb"""
            
            #si (intervention_1_start_time <= time, cdt1_1 = 1
            cdt1_1 =  m.Intermediate(m.if2(-intervention_1_start_time + time,0,1))
            #si time < intervention_1_start_time + intervention_1_delta, cdt1_2=1
            cdt1_2 =  m.Intermediate(m.if2(time - intervention_1_start_time + intervention_1_delta,1,0))
            #si (intervention_2_start_time <= time, cdt2_1 = 1
            cdt2_1 =  m.Intermediate(m.if2(-intervention_2_start_time + time,0,1))
            #si time < intervention_2_start_time + intervention_1_delta, cdt2_2=1
            cdt2_2 =  m.Intermediate(m.if2(time - intervention_2_start_time + intervention_2_delta,1,0))

            cdt1 =  m.Intermediate(cdt1_1 + cdt1_2) #est ==2 si t est dans l'intervalle
            cdt2 =  m.Intermediate(cdt2_1 + cdt2_2 )#est == 2 si t est dans l'intervalle
            m.Equation (cdt1 + cdt2 <4)
    

    print('\tok')


    #############################################################
    ########################objectif#############################
    #############################################################
    """Compute objectives (mean and expected excess)"""
    
    ############objectif1   
    """Compute objectives (mean)"""
     
    horizon_end = Instance[T_STR]
    S = Instance[SCENARIO_NUMBER]  #nb de scenarios par time step 
    print("S = ",S)
    print("horizon_end = ",horizon_end)
    alpha = 0.5
    liste_risk_s_t = [[m.Intermediate(m.sum([m.sum(
                        [x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
                        for st in range(1,int(intervention[TMAX_STR])+1) if str(t) in intervention[RISK_STR] and str(st) in intervention[RISK_STR][str(t)]])
                        for intervention_name, intervention in Interventions.items() ]))
                        for sc in range(1,int(S[t-1])+1) ] for t in range(1,int(horizon_end)+1) ]
    obj1 =  m.Intermediate(0)
    for t in range(1,horizon_end+1):
        obj1 +=  m.Intermediate((1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) )
    obj1 =  m.Intermediate((1/T)*obj1 )
    
    
    #obj1 = (1/T)* m.sum([ (1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) )]) 
    # ##
    # obj1 = (1/T)* m.sum([() for t in range(1,T+1)])        
    # () = (1/S[t])* m.sum([() for sc in range(1,S[t]) ])
    # () = m.sum([() for intervention_name, intervention in Interventions.items() ])
    # () = m.sum([( ) for st in range(1,T+1) ])
    # () = x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
    
    ##
    #m.Obj( obj1)
    
    
    
    
    """Compute risk distributions for all time steps, given the interventions starting times"""
    # liste_risk_s_t = [[m.sum([m.sum(
                        # [x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
                        # for st in range(1,int(intervention[TMAX_STR])+1) ])
                        # for intervention_name, intervention in Interventions.items() ])
                        # for sc in range(1,S[t-1]+1) ] for t in range(1,horizon_end+1) ]

    ############objectif2
    
    """Compute objectives (mean and expected excess)"""
    
    obj2  = 0
    Q_t = 0
    """
    for t in range(1,horizon_end+1):
        # quantile #le piem element quand on range l'ensemble dans l'odre croissant
            #calcul de p
        p = Instance[QUANTILE_STR]
        p = p*S[t-1]
        if p-int(p)!=0: 
            p = int(p)+1
        
        print("p = ",p)
        p = int(p)
        #index = p-1
            # trouver le p_ieme min 
        print("len() = ",S[t-1]+1)
        E = [ liste_risk_s_t[t-1][sc-1] for sc in range(1,S[t-1]+1)  ]
        print("le tri")
        for i in range (p):
            l_new = [elt for elt in E] 
            min = E[i]
            pos_min = i
            for j in range(i, len(E)):
                temp = m.if3( E[j]-min,E[j],min)
                min = temp
                temp = m.if3( E[j]-min,j,pos_min )
                pos_min = temp 
            # le min est trouve 
            # on echange les positions de i et pos_min 
            temp = E[j]
            E[i]=E[j] 
            E[j] = temp 
            #E[i], E[j] = E[j],E[i] 
            
        Q_t = E[p-1]
    
            
            
        

        print("Q_t = ",Q_t)
        
        
        
        #Compute risk distributions for all time steps, given the interventions starting times
        risk_t= (1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) 
        
        #calul de l'objectif 2
        obj2 += m.max2(0, Q_t - risk_t)
    """
    obj2 = (1/T)*obj2
    
    print("obj ",obj2)
    
    m.Obj(alpha*obj1 + (1-alpha)*obj2)
    
    #solution 
    m.options.SOLVER=1#pour des solututions entieres
    print("le max = ",m.options.MAX_ITER)
    #m.options.MAX_ITER = 100000
    m.options.MAX_MEMORY = 50000
    

    
    m.solve(disp=True,debug=True)
    
    
    #apm_get(server,app,'infeasibilities.txt')
    #m.open_folder()
    print(x)
    out = open( os.path.join(directory,'output_'+base+'.txt'),"w")
    for intervention, liste in x.items():
        for i in range(len(liste)):
            elt = liste[i]
            if int(elt[0])==1:
                print(intervention," commence à : t=",i+1 )
                out.writelines( [intervention +  ' ' + str(i+1)+'\n'])
    
    print("le max = ",m.options.MAX_ITER)
    
    out.close()
    

#prend le chemin vers le ichier json
def solus2p(path):
    os.chdir(dir1)
    instance_file = os.path.abspath(path)
    #print(1)
    #print("instance_file = ", instance_file[2:4], "type = ",type(instance_file))


    #creer un dossier dont le nom est celui diu fichier json 
    base=os.path.basename(instance_file)
    base = os.path.splitext(base)
    if base[1]!='json': print('ERROR: expecting a json file ')
    base  = base[0]
    directory = os.path.abspath(os.path.join('_AvecObj1_2p_solver1',base))
    if not os.path.exists(directory):
        os.makedirs(directory)

    
    
    
    resolution(instance_file,directory,base)
    
    os.chdir(dir1)

## Avec objectif1; no shutdown; solus2, "AvecObj1"

In [29]:
solus2p("files\\A_01.json")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



	resources constraints...
exclusions constraints...
	ok
('S = ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
('horizon_end = ', 90)
('obj ', 0)
('le max = ', 250)
 ----------------------------------------------------------------
 APMonitor, Version 0.9.2
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 

('Error:', 'At line 463 of file custom.f90\nTraceback: not available, compile with -ftrace=frame or -ftrace=full\nOperating system error: Impossible de cr\xe9er un fichier d\xe9j\xe0 existant.\n\nMemory allocation failed\n')
Error: 'results.json' not found. Check above for additional error details


IOError: [Errno 2] No such file or directory: 'c:\\users\\herman~1.ago\\appdata\\local\\temp\\tmpy6ex3egk_model8\\options.json'

### Le programme, après un nombre maximal d'itérations, n'a pas réussi à converger et a eu un beug. 

In [4]:
#solus3("files\\example1.json")

# On tient compte de l'objectif 2 mais on utilise un solveur NLP pour limiter le nombre d'itérations

In [20]:
# -*- coding: utf-8 -*-
"""
@author: Groupe formé de 
Akram IBN EL AHMER​
Hermann LEIBNITZ KLAUS AGOSSOU​
Reda ASSIOUI​
Olivier DANIEL STEPHANE KOMBA MBONGO​
Souad EZZOUINE​
"""
import os
import sys
import numpy as np
import json
from gekko import GEKKO


os.chdir(dir1)



####################
### Utils ##########
####################

## Global variables
CUR_DIR = os.getcwd()
PAR_DIR = os.path.dirname(CUR_DIR)
RESOURCES_STR = 'Resources'
SEASONS_STR = 'Seasons'
INTERVENTIONS_STR = 'Interventions'
EXCLUSIONS_STR = 'Exclusions'
T_STR = 'T'
SCENARIO_NUMBER = 'Scenarios_number'
RESOURCE_CHARGE_STR = 'workload'
TMAX_STR = 'tmax'
DELTA_STR = 'Delta'
MAX_STR = 'max'
MIN_STR = 'min'
RISK_STR = 'risk'
START_STR = 'start'
QUANTILE_STR = "Quantile"
ALPHA_STR = "Alpha"

## Json reader
def read_json(filename):
    """Read a json file and return data as a dict object"""

    print('Reading json file ' + filename + '...')
    f = open(filename, 'r')
    Instance = json.load(f)
    f.close()
    print('Done')
    print("Instance = ",Instance)
    return Instance





def resolution(instance_file,directory,out):
    # Read Instance
    Instance = read_json(instance_file)
    os.chdir(directory)
    m = GEKKO(remote=False)
    T=Instance[T_STR] #nb  de time steps 
    
    #variables 
    #ni = 3 #nb d'interventions 
    nj = int(Instance['T']) #nb de start_time 
    x = {}
    
    Interventions = Instance[INTERVENTIONS_STR]
    
    #############################################################
    ########################variables############################
    #############################################################
    
    
    #definie les variables 
    for intervention_name, intervention in Interventions.items():
        x[intervention_name] = [m.Var(lb=0,ub=1,integer=True,name="{}_t_{}".format(intervention_name,j)) for j in range(nj)]
    #les stocker dans le dictionnaire 
    for intervention_name, intervention in Interventions.items():
        #start_time 
        Interventions[intervention_name][START_STR] = m.Intermediate(sum ([ m.Intermediate((st+1)* x[intervention_name][st]) for st in range( len(x[intervention_name])) ]))
        
    #############################################################
    ########################contraintes##########################
    #############################################################
    #contraintes 
        #for each intervention: 
            #- 1 <= start_time <= horizon_end=instance["T"]
            #- int(intervention[TMAX_STR]) >= start_time
    horizon_end = int(Instance[T_STR])
    
    """ schedule constraints"""
    
    #contrainte 1 #qq i, 1 <= start_time <= horizon_end=instance["T"]
    Interventions = Instance[INTERVENTIONS_STR]
    
    for intervention_name, intervention in Interventions.items():
        #il y a doit y avoir un start time  
        print("intervention est bien de type: ",type(intervention))
        #cdt #qq i, 1 <= start_time <= horizon_end=instance["T"]
        s = m.Intermediate(0,name="s for {}".format(intervention_name))
        for st in range(1,horizon_end+1):
            s+= x[intervention_name][st-1]
        print("ici, intervention est bien de type: ",type(intervention))
        #m.Equation(s==1)
        m.Equation(s<=1.2)
        m.Equation(0.8<=s)
                
        print("encore, intervention est bien de type: ",type(intervention))
        #j'ajoute apres cette contrainte à travers une contrainte de type inegalite
        
        #cdt #qq i, 1 <= start_time + delta_i_t<= horizon_end=instance["T"] +1
        # ... retrieve their respective starting times...
        start_time = m.Intermediate(sum ([ (st+1)* x[intervention_name][st] for st in range( len(x[intervention_name])) ]), name = "st of {}".format(intervention_name))
        # ... and their respective deltas (duration)
        intervention_delta = m.Intermediate(0)
        print("la, intervention est bien de type: ",type(intervention))
        for st in range(1,int(intervention[TMAX_STR])+1):
            intervention_delta+=m.Intermediate(x[intervention_name][st-1]*int(intervention[DELTA_STR][st - 1]))
        #intervention_delta = m.Intermediate(sum( [ m.Intermediate(x[intervention_name][st-1]*int(intervention[DELTA_STR][st - 1]))
                                    #for st in range(1,int(intervention[TMAX_STR])+1) ])) # get index in list
        m.Equation( start_time + intervention_delta<= Instance[T_STR]+1)
        m.Equation ( 1<=start_time) #contrainte: il debute pas à st==0 #meilleure contraonte que l'egalite
        #contrainte2 #qq i, int(intervention[TMAX_STR]) >= start_time
        m.Equation (  start_time <= intervention[TMAX_STR] ) #contrainte2 #qq i, int(intervention[TMAX_STR]) >= start_time
    
            
    
    for intervention,liste in x.items():
        for var in liste:
            #cdt1==1 si pres de 0 sinon cdt1==0
            cdt1 = m.Intermediate(m.if3(-0.05-var,0,1) * m.if3(var-0.05,0,1))
            #cdt2==1 si pres de 1 sinon cdt2==0
            cdt2 = m.Intermediate(m.if3(0.95-var,0,1) * m.if3(var-1.05,0,1))
            #soit pres de 1 soit pres de 0
            m.Equation(cdt1+cdt2==1)
    
    """resources constraints"""
    
    print('\tresources constraints...')
    #contrainte3 
    #horizon_end = Instance[T_STR]
    """Compute effective workload (i.e. resources consumption values) for every resource and every time step"""
    
    # tolerance=1e-5
    # print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    # for c, constraint in Instance[RESOURCES_STR].items():
        # for t in range(1,horizon_end+1):
            # ss = sum ([ sum([ x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    # for st in range(1,intervention[TMAX_STR]+1) ])
                    # for intervention_name, intervention in Interventions.items() 
                    # ])
            # m.Equation(  constraint[MAX_STR][t-1] + tolerance  >= ss)
            # m.Equation( ss  >= constraint[MIN_STR][t-1] - tolerance )
            
    tolerance=m.Const(1e-5, name = "tolerance")
    #print(sum([ intervention["workload"]["c1"]["1"]["1"] for intervention_name, intervention in Interventions.items()]))
    for c, constraint in Instance[RESOURCES_STR].items():
        print("ressource = ",c)
        for t in range(1,horizon_end+1):
            ss=m.Intermediate(0,name="ss_c_{}_t_{}".format(c,t))
            print("c = {} et t = {}".format(c,t))
            for intervention_name, intervention in Interventions.items():
                for st in range(1,int(intervention[TMAX_STR])+1):
                    if c in intervention["workload"] and str(t) in intervention["workload"][c] and str(st) in intervention["workload"][c][str(t)]:
                        a = m.Intermediate(x[intervention_name][st-1])
                        
                        b = m.Intermediate(intervention["workload"][c][str(t)][str(st)])
                        ss+=a*b
                        #ss+= x[intervention_name][st-1] * intervention["workload"][c][str(t)][str(st)]
                    
            m.Equation(ss>= constraint[MIN_STR][t-1] - tolerance )
            m.Equation(constraint[MAX_STR][t-1] + tolerance >= ss)
            
    #m.Equation(resource[MAX_STR][t-1] + tolerance  >= resources_usage[resource_name][t-1])
    

    """exclusions constraints"""
    
    print('exclusions constraints...')
    # Retrieve Interventions and Exclusions
    Interventions = Instance[INTERVENTIONS_STR]
    Exclusions = Instance[EXCLUSIONS_STR]
    # Assert every exclusion holds
    for exclusion in Exclusions.values():
        # Retrieve exclusion infos
        [intervention_1_name, intervention_2_name, season] = exclusion
        # Retrieve concerned interventions...
        intervention_1 = Interventions[intervention_1_name]
        intervention_2 = Interventions[intervention_2_name]
        # ... their respective starting times...
        intervention_1_start_time = sum ([ (st+1)* x[intervention_1_name][st] for st in range( len(x[intervention_1_name])) ])
        intervention_2_start_time = sum ([ (st+1)* x[intervention_2_name][st] for st in range( len(x[intervention_2_name])) ])
        # ... and their respective deltas (duration)
        intervention_1_delta = sum( [x[intervention_1_name][st-1]*int(intervention_1[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_1[TMAX_STR])+1) ]) # get index in list
        #intervention_1_delta = int(intervention_1[DELTA_STR][intervention_1_start_time - 1]) # get index in list
        intervention_2_delta = sum( [x[intervention_2_name][st-1]*int(intervention_2[DELTA_STR][st - 1])
                                    for st in range(1,int(intervention_2[TMAX_STR])+1) ]) # get index in list
        # Check overlaps for each time step of the season 
        for time_str in Instance[SEASONS_STR][season]:
            time = int(time_str)
            #equation       
            #si (intervention_1_start_time <= time, cdt1_1 = 1
            cdt1_1 = m.if2(-intervention_1_start_time + time,0,1)
            #si time < intervention_1_start_time + intervention_1_delta, cdt1_2=1
            cdt1_2 = m.if2(time - intervention_1_start_time + intervention_1_delta,1,0)
            #si (intervention_2_start_time <= time, cdt2_1 = 1
            cdt2_1 = m.if2(-intervention_2_start_time + time,0,1)
            #si time < intervention_2_start_time + intervention_1_delta, cdt2_2=1
            cdt2_2 = m.if2(time - intervention_2_start_time + intervention_2_delta,1,0)

            cdt1 = cdt1_1 + cdt1_2 #est ==2 si t est dans l'intervalle
            cdt2 = cdt2_1 + cdt2_2 #est == 2 si t est dans l'intervalle
            m.Equation (cdt1 + cdt2 <4)
    
    
    print('\tok')

    #############################################################
    ########################objectif#############################
    #############################################################
    """Compute objectives (mean and expected excess)"""
    
    ############objectif1   
    """Compute objectives (mean)"""
    T=Instance[T_STR] #nb  de time steps 
    horizon_end = Instance[T_STR]
    S = Instance[SCENARIO_NUMBER]  #nb de scenarios par time step 
    print("S = ",S)
    print("horizon_end = ",horizon_end)
    alpha = Instance[ALPHA_STR]
    liste_risk_s_t = [[m.Intermediate(m.sum([m.sum(
                        [x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
                        for st in range(1,int(intervention[TMAX_STR])+1) if str(t) in intervention[RISK_STR] and str(st) in intervention[RISK_STR][str(t)]])
                        for intervention_name, intervention in Interventions.items() ]))
                        for sc in range(1,int(S[t-1])+1) ] for t in range(1,int(horizon_end)+1) ]
    obj1 = 0
    for t in range(1,horizon_end+1):
        obj1 += m.Intermediate((1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) )
    obj1 = (1/T)*obj1 
    
    
    #obj1 = (1/T)* m.sum([ (1/S[t-1]) * m.sum( [ liste_risk_s_t[t-1][sc-1] for sc in range (1,S[t-1]+1)   ] ) )]) 
    # ##
    # obj1 = (1/T)* m.sum([() for t in range(1,T+1)])        
    # () = (1/S[t])* m.sum([() for sc in range(1,S[t]) ])
    # () = m.sum([() for intervention_name, intervention in Interventions.items() ])
    # () = m.sum([( ) for st in range(1,T+1) ])
    # () = x[intervention_name][st-1] * intervention[RISK_STR][str(t)][str(st)][sc-1]
    
    ##
    #m.Obj( obj1)
    obj2=0
    
    m.Obj(alpha*obj1 + (1-alpha)*obj2)
    
    #solution 
    m.options.SOLVER=3#pour des solututions entieres
    print("le max = ",m.options.MAX_ITER)
    #m.options.MAX_ITER = 100000
    m.options.MAX_MEMORY = 500000
    '''
    # optional solver settings with APOPT
    m.solver_options = ['minlp_maximum_iterations 100000', \
                        # minlp iterations with integer solution
                        'minlp_max_iter_with_int_sol 10', \
                        # nlp sub-problem max iterations
                        'nlp_maximum_iterations 50', \
                        # find integer solution faster
                        'minlp_branch_method 1'
                        # ecart par rapport à l'entier 
                        'minlp_integer_tol 0.2']
    '''
    '''
    m.solver_options = ['minlp_maximum_iterations 100000', \
                        # minlp iterations with integer solution
                        'minlp_max_iter_with_int_sol 10', \
                        # nlp sub-problem max iterations
                        'nlp_maximum_iterations 50', \
                        # find inegr solution faster
                        'minlp_branch_method 1', \
                        # covergence tolerance
                        'minlp_gap_tol 0.00']
    '''
    
    #m.solve(disp=True,debug=True)
    m.solve(disp=True,debug=True)
    #apm_get(server,app,'infeasibilities.txt')
    #m.open_folder()
    print(x)


    for intervention, liste in x.items():
        for i in range(len(liste)):
            elt = liste[i]
            if int(elt[0])==1:
                print(intervention," : t",i+1 )
                out.writelines( [intervention +  ' ' + str(i+1)+'\n'])

    print("le max = ",m.options.MAX_ITER)
    out.close()

#prend le chemin vers le ichier json
def solus4(path):
    os.chdir(dir1)
    instance_file = os.path.abspath(path)
    #print(1)
    #print("instance_file = ", instance_file[2:4], "type = ",type(instance_file))


    #creer un dossier dont le nom est celui diu fichier json 
    base=os.path.basename(instance_file)
    base = os.path.splitext(base)
    if base[1]!='json': print('ERROR: expecting a json file ')
    base  = base[0]
    directory = os.path.abspath(os.path.join('_obj12_avecsolveur3',base))
    if not os.path.exists(directory):
        os.makedirs(directory)


    out = open( os.path.join(directory,'output_'+base+'.txt'),"w")
    
    resolution(instance_file,directory,out)
    
    os.chdir(dir1)

# Avec obj 1 et 2 en NLP; # no shutdown, solus4, "avecsolveur2"

In [21]:
#solus4("files\\A_01.json")

In [4]:
import os
os.startfile('.')